In [13]:
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import torch

!ls "/content/drive/My Drive/Colab_Notebooks/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 agnostic_bert_article_embeddings.pkl	 datacluster_3_with_embeddings.pkl
 agnostic_bert_first100_embeddings.pkl	 datacluster_4_with_embeddings.pkl
 agnostic_bert_title_embeddings10k.pkl	 dbscan_Agnostic_clusters_analysis.csv
 agnostic_bert_title_embeddings.pkl	 full_clusterd.pkl
 all-the-news-2-1.csv			 full_clustered.pkl
 bert_article_embeddings10K.pkl		 full_data_fixed_dates.pkl
 bert_article_embeddings.pkl		 full_dataset_Fine_embeddings.pkl
 bert_first100_embeddings.pkl		 full_dataset_with_embeddings.pkl
 bert_title_embeddings10K.pkl		 full_data_with_dbscan.pkl
 cleaned_100k.csv			 full_data_with_dbscan_revised.pkl
 cleaned_10k.csv			'full_data_with_finetuned_embeddings&dbscan.pkl'
 cleaned_10k_with_first100.csv		 full_data_with_finetuned_embeddings.pkl
 cleaned_15k.csv			 Full_data_with_finetuned_embeddings.pkl
 cleaned_30k.csv			 full_data_with_finetuned

In [ ]:
df = pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/New_Normalized_Finetuned.pkl')

In [15]:

from google.colab import drive
import pandas as pd
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
import re

drive.mount('/content/drive')

!ls "/content/drive/My Drive/Colab_Notebooks/data"
df = pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/New_Normalized_Finetuned.pkl')

# Function to check if a string is likely English
def is_english(text):
  if not isinstance(text, str):
      return False
  return bool(re.search(r'[a-zA-Z]', text))

# Filter out non-English titles and F100
df = df[df['title'].apply(is_english)]
df = df[df['F100'].apply(is_english)]

df.to_pickle('/content/drive/My Drive/Colab_Notebooks/data/New_Normalized_Finetuned.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 agnostic_bert_article_embeddings.pkl	 datacluster_3_with_embeddings.pkl
 agnostic_bert_first100_embeddings.pkl	 datacluster_4_with_embeddings.pkl
 agnostic_bert_title_embeddings10k.pkl	 dbscan_Agnostic_clusters_analysis.csv
 agnostic_bert_title_embeddings.pkl	 full_clusterd.pkl
 all-the-news-2-1.csv			 full_clustered.pkl
 bert_article_embeddings10K.pkl		 full_data_fixed_dates.pkl
 bert_article_embeddings.pkl		 full_dataset_Fine_embeddings.pkl
 bert_first100_embeddings.pkl		 full_dataset_with_embeddings.pkl
 bert_title_embeddings10K.pkl		 full_data_with_dbscan.pkl
 cleaned_100k.csv			 full_data_with_dbscan_revised.pkl
 cleaned_10k.csv			'full_data_with_finetuned_embeddings&dbscan.pkl'
 cleaned_10k_with_first100.csv		 full_data_with_finetuned_embeddings.pkl
 cleaned_15k.csv			 Full_data_with_finetuned_embeddings.pkl
 cleaned_30k.csv			 full_data_with_finetuned

In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances


# טווחי המרחקים לבדיקה
distance_ranges = [
    (0.02, 0.03),
    (0.03, 0.04),
    (0.05, 0.06)

]

# מעבר על כל קלאסטר ראשי
for cluster_num in df['ag_title_cluster'].unique():
    print(f"\n🔍 Analyzing Cosine Similarity for Main Cluster {cluster_num}")

    # סינון הנתונים של הקלאסטר
    cluster_df = df[df['ag_title_cluster'] == cluster_num]

    if len(cluster_df) < 2:
        print(f"⚠️ Cluster {cluster_num} has less than 2 articles, skipping.")
        continue

    # טעינת ה-embeddings המנורמלים
    embeddings = np.stack(cluster_df['F100_finetuning_embedding_normalized'].values)

    # חישוב מטריצת Cosine Distance
    distance_matrix = cosine_distances(embeddings)

    # מילון לאחסון זוגות דומים מכל טווח מרחקים
    selected_pairs = {str(b): [] for b in distance_ranges}

    # מעבר על כל זוג כתבות
    num_articles = len(cluster_df)
    for i in range(num_articles):
        for j in range(i + 1, num_articles):  # מונעים כפילות
            cos_distance = distance_matrix[i, j]

            # בדיקה אם המרחק נכנס לטווחים שנבחרו
            for b in distance_ranges:
                if b[0] <= cos_distance < b[1]:
                    if len(selected_pairs[str(b)]) < 5:  # ניקח 5 דוגמאות בלבד מכל טווח
                        selected_pairs[str(b)].append((i, j, cos_distance))
                    break  # יציאה מהלולאה אחרי מציאת טווח מתאים

    # **הדפסת דוגמאות שנבחרו מכל טווח מרחקים**
    for b, pairs in selected_pairs.items():
        print(f"\n📌 Distance Range: {b} - Showing 2 Examples")
        for i, j, cos_dist in pairs:
            title1 = cluster_df.iloc[i]['title'][:100]
            article1 = cluster_df.iloc[i]['F100'][:200] + "..."
            title2 = cluster_df.iloc[j]['title'][:100]
            article2 = cluster_df.iloc[j]['F100'][:200] + "..."
            print(f"\n🔹 Cosine Distance: {cos_dist:.4f}")
            print(f"📄 title 1: {title1}")
            print(f"📄 Article 1: {article1}")
            print(f"📄 title 2: {title2}")
            print(f"📄 Article 2: {article2}")


🔍 Analyzing Cosine Similarity for Main Cluster 0

📌 Distance Range: (0.02, 0.03) - Showing 2 Examples

🔹 Cosine Distance: 0.0238
📄 title 1: RPT-COLUMN-Nickel bubble deflates but a bear trap may be opening: Andy Home
📄 Article 1: (Repeats Thursday column with no changes to text) * (The opinions expressed here are those of the author, a columnist for Reuters.) By Andy Home LONDON, Dec 5 (Reuters) - The nickel price bubble is sl...
📄 title 2: COLUMN-Nickel bubble deflates but a bear trap may be opening: Andy Home
📄 Article 2: (The opinions expressed here are those of the author, a columnist for Reuters.) * LME nickel market: tmsnrt.rs/2Pc2vPo By Andy Home LONDON, Dec 5 (Reuters) - The nickel price bubble is slowly deflatin...

🔹 Cosine Distance: 0.0283
📄 title 1: Dancing to (and Around) Bach
📄 Article 1: #SpeakinginDance Watch a portion of “New Work for Goldberg Variations,” a collaboration between Pam Tanowitz and Simone Dinnerstein. #SpeakingInDance is a weekly visual exploration of da

In [20]:
# prompt: create 'date' colomn from 'year' 'month' 'day' and save

# Assuming 'year', 'month', and 'day' columns exist in your DataFrame 'df'

df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

# Save the updated DataFrame
df.to_pickle('/content/drive/My Drive/Colab_Notebooks/data/New_Normalized_Finetuned.pkl')


In [26]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
import random

# Load the dataset
df = pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/New_Normalized_Finetuned.pkl')

# Define the cosine similarity threshold
threshold = 0.2

# Initialize an empty list to store the selected pairs
selected_pairs = []

# Iterate over all main clusters
for cluster_num in df['ag_title_cluster'].unique():
    print(f"\n🔍 Analyzing Cosine Similarity for Main Cluster {cluster_num}")

    # Filter data for the current cluster
    cluster_df = df[df['ag_title_cluster'] == cluster_num]

    if len(cluster_df) < 2:
        print(f"⚠️ Cluster {cluster_num} has less than 2 articles, skipping.")
        continue

    # Load normalized embeddings
    embeddings = np.stack(cluster_df['F100_finetuning_embedding_normalized'].values)

    # Calculate cosine similarity matrix
    similarity_matrix = 1 - cosine_distances(embeddings)  # Convert distance to similarity

    # Create a list of all valid pairs in the cluster
    potential_pairs = []
    num_articles = len(cluster_df)
    for i in range(num_articles):
        for j in range(i + 10, num_articles):
            cos_similarity = similarity_matrix[i, j]

            # Check if the cosine similarity is below the threshold
            if cos_similarity < threshold:
                potential_pairs.append((i, j, cos_similarity))

    # Select up to 10 random pairs if there are enough
    if len(potential_pairs) > 10:
        selected = random.sample(potential_pairs, 10)
    else:
        selected = potential_pairs

    # Store the selected pairs with full information
    for i, j, cos_similarity in selected:
        original_index1 = cluster_df.index[i]
        original_index2 = cluster_df.index[j]

        selected_pairs.append([
            cluster_num,
            df.loc[original_index1]['date'] if 'date' in df.columns else "N/A",  # Date 1
            df.loc[original_index1]['title'][:100],  # Title 1
            df.loc[original_index1]['F100'][:200] + "...",  # F100 Text 1
            df.loc[original_index2]['date'] if 'date' in df.columns else "N/A",  # Date 2
            df.loc[original_index2]['title'][:100],  # Title 2
            df.loc[original_index2]['F100'][:200] + "...",  # F100 Text 2
            cos_similarity  # Cosine similarity
        ])

# Create a DataFrame from the selected pairs
pairs_df = pd.DataFrame(selected_pairs, columns=['cluster', 'date1', 'title1', 'F100_1', 'date2', 'title2', 'F100_2', 'cosine_similarity'])

# Save the DataFrame to a CSV file
pairs_df.to_csv('/content/drive/My Drive/Colab_Notebooks/data/similar_article_pairs_with_dates.csv', index=False)

print("✅ CSV file with 10 random pairs per cluster created successfully!")


🔍 Analyzing Cosine Similarity for Main Cluster 0

🔍 Analyzing Cosine Similarity for Main Cluster 2

🔍 Analyzing Cosine Similarity for Main Cluster 5

🔍 Analyzing Cosine Similarity for Main Cluster 1

🔍 Analyzing Cosine Similarity for Main Cluster 3

🔍 Analyzing Cosine Similarity for Main Cluster 4
✅ CSV file with 10 random pairs per cluster created successfully!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 agnostic_bert_article_embeddings.pkl	 datacluster_3_with_embeddings.pkl
 agnostic_bert_first100_embeddings.pkl	 datacluster_4_with_embeddings.pkl
 agnostic_bert_title_embeddings10k.pkl	 dbscan_Agnostic_clusters_analysis.csv
 agnostic_bert_title_embeddings.pkl	 full_clusterd.pkl
 all-the-news-2-1.csv			 full_clustered.pkl
 bert_article_embeddings10K.pkl		 full_data_fixed_dates.pkl
 bert_article_embeddings.pkl		 full_dataset_Fine_embeddings.pkl
 bert_first100_embeddings.pkl		 full_dataset_with_embeddings.pkl
 bert_title_embeddings10K.pkl		 full_data_with_dbscan.pkl
 cleaned_100k.csv			 full_data_with_dbscan_revised.pkl
 cleaned_10k.csv			'full_data_with_finetuned_embeddings&dbscan.pkl'
 cleaned_10k_with_first100.csv		 full_data_with_finetuned_embeddings.pkl
 cleaned_15k.csv			 Full_data_with_finetuned_embeddings.pkl
 cleaned_30k.csv			 full_data_with_finetuned

#sameday

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity

# טעינת הנתונים המלאים עם תאריכים
df = pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/full_data_with_finetuned_embeddings.pkl')

# **ניקוי התאריכים - הפיכת year, month, day למספרים שלמים**
df['year'] = df['year'].astype('Int64', errors='ignore')
df['month'] = df['month'].astype('Int64', errors='ignore')
df['day'] = df['day'].astype('Int64', errors='ignore')

# טיפול בערכים חסרים על ידי הצבת תאריך ברירת מחדל
df.fillna({'year': 2000, 'month': 1, 'day': 1}, inplace=True)

# פונקציה לחישוב הפרש בתאריכים בצורה בטוחה
def date_difference(row1, row2):
    try:
        date1 = pd.to_datetime(f"{int(row1['year'])}-{int(row1['month'])}-{int(row1['day'])}", errors='coerce')
        date2 = pd.to_datetime(f"{int(row2['year'])}-{int(row2['month'])}-{int(row2['day'])}", errors='coerce')

        # אם אחד מהתאריכים לא תקין, נחזיר הפרש גבוה כדי לא להתייחס אליו כאל זהה
        if pd.isna(date1) or pd.isna(date2):
            return 9999

        return abs((date1 - date2).days)

    except Exception as e:
        print(f"⚠️ Error parsing dates: {row1['year']}-{row1['month']}-{row1['day']} | {row2['year']}-{row2['month']}-{row2['day']}")
        return 9999

# מעבר על כל קלאסטר ראשי
for cluster_num in df['ag_title_cluster'].unique():
    print(f"\n🔍 Analyzing Cosine Similarity for Main Cluster {cluster_num}")

    # סינון הנתונים של הקלאסטר
    cluster_df = df[df['ag_title_cluster'] == cluster_num]

    if len(cluster_df) < 2:
        print(f"⚠️ Cluster {cluster_num} has less than 2 articles, skipping.")
        continue

    # טעינת ה-embeddings המנורמלים
    embeddings = np.stack(cluster_df['finetuning_embedding'].values)

    # חישוב מטריצת Cosine Similarity
    similarity_matrix = cosine_similarity(embeddings)

    # אוסף זוגות לפי טווחים
    similarity_ranges = {
        "0 - 0.12": [],
        "0.03 - 0.05": [],
        "0.05 - 0.07": [],
        "0.07 - 0.09": [],
        "0.8+": []
    }

    # מעבר על כל זוג כתבות בתוך הקלאסטר
    index_pairs = list(combinations(range(len(cluster_df)), 2))

    for i, j in index_pairs:
        similarity = similarity_matrix[i, j]
        cosine_distance = 1 - similarity  # המרחק הקוסינוסי

        # מציאת טווח המתאים
        if 0.1 <= cosine_distance < 0.12:
            key = "0 - 0.12"
        elif 0.12 <= cosine_distance < 0.15:
            key = "0.03 - 0.05"
        elif 0.15 <= cosine_distance < 0.18:
            key = "0.05 - 0.07"
        elif 0.18 <= cosine_distance < 0.22:
            key = "0.07 - 0.09"
        elif cosine_distance >= 0.8:
            key = "0.8+"
        else:
            continue

        # בדיקת הפרש תאריכים
        row1, row2 = cluster_df.iloc[i], cluster_df.iloc[j]
        days_diff = date_difference(row1, row2)

        # אם ההפרש בתאריכים קטן, יש עדיפות לזוג הזה
        similarity_ranges[key].append((i, j, cosine_distance, days_diff))

    # הצגת 3 דוגמאות לכל טווח
    for key, pairs in similarity_ranges.items():
        if not pairs:
            continue

        # מיון לפי הפרש ימים קטן
        pairs = sorted(pairs, key=lambda x: x[3])[:3]

        print(f"\n📌 Showing 3 Examples for Cosine Distance Range: {key}")
        for i, j, dist, days_diff in pairs:
            row1, row2 = cluster_df.iloc[i], cluster_df.iloc[j]
            print(f"\n🔹 Cosine Distance: {dist:.4f}, Date Diff: {days_diff} days")
            print(f"📄 Article 1: {row1['F100'][:150]}...")
            print(f"📄 Article 2: {row2['F100'][:150]}...")


🔍 Analyzing Cosine Similarity for Main Cluster 0

📌 Showing 3 Examples for Cosine Distance Range: 0 - 0.12

🔹 Cosine Distance: 0.1169, Date Diff: 0 days
📄 Article 1: New research from the National Institutes of Health (NIH) has found that the use of permanent hair dye and chemical hair straighteners were positively...
📄 Article 2: The common practice of coloring hair may lead to an increased risk of breast cancer, new research finds. In a study published Wednesday, researchers a...

🔹 Cosine Distance: 0.1045, Date Diff: 0 days
📄 Article 1: The trailer for the new James Bond film, No Time to Die, is finally here—and it looks like everything you'd want from a Bond movie. Explosions! Aston ...
📄 Article 2: (CNN)Holy crap on a cracker Daniel Craig! The new trailer for the latest James Bond film, "No Time to Die" debuted Wednesday and boy is it meant to ex...

🔹 Cosine Distance: 0.1056, Date Diff: 0 days
📄 Article 1: You are an alien astronomer living 4.5 billion years from now. The Sun th

KeyboardInterrupt: 

<ipython-input-6-99faae88a81a>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['date'].fillna(pd.to_datetime("2000-01-01"), inplace=True)


✅ Fixed date column - 0 invalid dates replaced.

🎉 Dates fixed and saved successfully!
✅ Data saved at: /content/drive/My Drive/Colab_Notebooks/data/full_data_with_finetuned_embeddings.pkl


At a distance of 0.05 - 0.09
	•	Most articles are identical, but some are not (e.g., in Cluster 0).
	•	This appears to be a borderline range – articles here are quite similar but may not necessarily report on the same event.
	2.	At a distance of 0.09 - 0.1
	•	About half of the pairs are marked as “identical” and half as “not identical.”
	•	This seems to be a critical threshold – the transition point between articles reporting on the same event and those covering different but related events.
	3.	At a distance of 0.1 - 0.11
	•	Almost all articles are not identical, with only a few exceptions.
	•	This suggests that beyond 0.1, most articles discuss similar topics but not the same event.
	4.	At a distance of 0.11 - 0.12
	•	Almost none of the pairs are identical – the distance is too high for articles reporting the same event.
	•	At this range, they seem to belong to the same general topic (e.g., politics, sports) but not the same event.

The estimated threshold between identical and non-identical articles is around 0.09 - 0.1.
💡 This means:
	•	Articles with a cosine distance ≤ 0.09 are likely reporting the same event.
	•	Articles with a distance > 0.1 discuss a different event.
	•	The range 0.09 - 0.1 is a transition zone, and we may need to verify these articles manually.